# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [37]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [38]:
csv_path="cities.csv"
cities_df=pd.read_csv(csv_path)
cities_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,0,Puerto Escondido,MX,2020-10-16 11:04:34,15.85,-97.07,78.01,75,26,4.52
1,1,Kavaratti,IN,2020-10-16 11:05:12,10.57,72.64,81.70,77,12,18.68
2,2,Saskylakh,RU,2020-10-16 11:05:12,71.92,114.08,10.94,95,88,4.50
3,3,Arrondissement de Dieppe,FR,2020-10-16 11:05:12,49.75,1.17,55.00,71,90,9.17
4,4,Puerto Ayora,EC,2020-10-16 11:05:12,-0.74,-90.35,70.00,90,94,3.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [39]:
gmaps.configure(api_key=g_key)

In [40]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity (%)"]

In [41]:
figure_layout = {
    'width': "100%",
    'height': '400px',
    'padding': '1px',}

fig = gmaps.figure(layout=figure_layout, center=(20.0, 5.0), zoom_level=1.9)
heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False)
fig.add_layer(heatmap_layer)
heatmap_layer.max_intensity = max(weights)
heatmap_layer.point_radius = 3
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
#Criteria: Temp between 70-80(f), wind<10(%),Cloud==0,
ideal_cities = cities_df.loc[(cities_df["Max Temp (F)"] > 70) & 
                               (cities_df["Max Temp (F)"] < 80) & 
                               (cities_df["Wind Speed (mph)"] < 10) & 
                               (cities_df["Cloudiness (%)"] == 0), :].reset_index(drop=True).dropna()
ideal_cities

,City ID,City,Country,Date,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,33,Buckeye,US,2020-10-16 11:03:56,33.37,-112.58,78.01,22,0,5.23
1,135,Kumluca,TR,2020-10-16 11:05:34,36.37,30.29,78.93,56,0,7.58
2,170,Kazygurt,KZ,2020-10-16 11:05:40,41.76,69.39,75.20,20,0,4.70
3,204,Sabzawār,IR,2020-10-16 11:02:32,36.21,57.68,71.60,13,0,9.17
5,237,Ardakān,IR,2020-10-16 11:02:32,32.31,54.02,75.20,6,0,6.93
6,243,Akdepe,TM,2020-10-16 11:05:52,42.06,59.38,75.20,18,0,6.93
7,394,Grand-Santi,GF,2020-10-16 11:06:18,4.25,-54.38,75.99,74,0,3.38
8,433,Pāveh,IR,2020-10-16 11:06:25,35.04,46.36,76.51,14,0,3.00
9,456,Seydi,TM,2020-10-16 11:06:32,39.48,62.91,76.08,18,0,8.03
10,466,Nurota,UZ,2020-10-16 11:02:11,40.56,65.69,75.20,15,0,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [43]:
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Buckeye,US,33.37,-112.58,NaN
1,Kumluca,TR,36.37,30.29,NaN
2,Kazygurt,KZ,41.76,69.39,NaN
3,Sabzawār,IR,36.21,57.68,NaN
5,Ardakān,IR,32.31,54.02,NaN
6,Akdepe,TM,42.06,59.38,NaN
7,Grand-Santi,GF,4.25,-54.38,NaN
8,Pāveh,IR,35.04,46.36,NaN
9,Seydi,TM,39.48,62.91,NaN
10,Nurota,UZ,40.56,65.69,NaN


In [44]:
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

params = { "keyword": target_search,
            "radius": target_radius,
            "type": target_type,
            "key": g_key}
#URL
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [45]:
for index, row in hotel_df.iterrows():
    
    # --- get the coordinates and store in params dictionary ---
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # --- call the API and get hotel details for each city ---
    hotel_data = requests.get(base_url, params).json()
    
    # --- use exception handling to save the first hotel name for each city ---
    
    try:
        hotel_name = hotel_data["results"][0]["name"]
        hotel_df.loc[index, "Hotel Name"] = hotel_name
        print(f"City: {row['City']}. Nearest hotel: {hotel_name}")
              
    except:
        print(f"Could not find information for city '{row['City']}'")    

City: Buckeye. Nearest hotel: Buckeye Motor Hotel
City: Kumluca. Nearest hotel: TurkuazKöy
Could not find information for city 'Kazygurt'
City: Sabzawār. Nearest hotel: Sabzevar Tourism hotel
City: Ardakān. Nearest hotel: Kheshtomah Ecolodge
Could not find information for city 'Akdepe'
City: Grand-Santi. Nearest hotel: Hotel La Villa Couacou
City: Pāveh. Nearest hotel: Eram Hotel
City: Seydi. Nearest hotel: Pidaraz mentler ketegi
City: Nurota. Nearest hotel: Nurota Hotel


In [46]:
hotel_df.dropna(how="any",inplace=True)
hotel_df.reset_index(drop=True, inplace=True)

#display final dataframe ---
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Buckeye,US,33.37,-112.58,Buckeye Motor Hotel
1,Kumluca,TR,36.37,30.29,TurkuazKöy
2,Sabzawār,IR,36.21,57.68,Sabzevar Tourism hotel
3,Ardakān,IR,32.31,54.02,Kheshtomah Ecolodge
4,Grand-Santi,GF,4.25,-54.38,Hotel La Villa Couacou
5,Pāveh,IR,35.04,46.36,Eram Hotel
6,Seydi,TM,39.48,62.91,Pidaraz mentler ketegi
7,Nurota,UZ,40.56,65.69,Nurota Hotel


In [47]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='400px', padding='1px', width='100%'))